In [27]:
#reference https://www.kaggle.com/c/trackml-particle-identification/discussion/57643

import os
from trackml.score  import score_event
from sklearn.manifold import TSNE
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import least_squares


In [28]:
# estimate helix paramater space
def helix_estimate_param_from_track(xyz):
    
    def residuals_xy(param, x, y):
        x0, y0 = param
        r = np.sqrt((x-x0)**2 + (y-y0)**2)
        d = r - r.mean()
        return d
    
    def residuals_z(param, r, z):
        m2,m1,m0 = param
        zz = m2*r**2 + m1*r + m0
        d = z - zz
        return d
    
    x = xyz[:3,0]
    y = xyz[:3,1]
    z = xyz[:3,2]
    
    param0 = (x.mean(), y.mean())
    res_lsq = least_squares(residuals_xy, param0, loss='soft_l1', f_scale=1.0, args=(x,y))
    x0,y0 = res_lsq.x
    r0 = np.sqrt((x-x0)**2 + (y-y0)**2).mean()
    r = np.sqrt(x**2 + y**2)
    
    if 1:
        param0 = (0,0,0)
        res_lsq = least_squares(residuals_z, param0, args=(r, z))
        m2, m1, m0 = res_lsq.x
        
    if 0:
        # polynomial fit of the degree 2, x**2 - quadradic 
        m2,m1,m0 = np.polyfit(r,z,2)
        
    param = (x0,y0,r0,m1,m2,m0)
    return param

# convert x,y,z to curve space
def helix_search_theta(xyz, param, S=2.5):
    
    x0, y0, r0, m2, m1, m0 = param
    theta0 = np.arctan2(y0, x0)
    #sort by z
    xyz = xyz[np.argsort(np.fabs(xyz[:,2]))]
    xx = xyz[:,0] - x0
    yy = xyz[:,1] - y0
    x = xx*np.cos(-theta0) - yy*np.sin(-theta0)
    y = xx*np.sin(-theta0) + yy*np.cos(-theta0)
    theta = np.arctan(y, -x)
    
    theta_min = 0
    theta_max = theta[-1]
    theta_num = S*len(xyz)*50
    return np.linspace(theta_min, theta_max, theta_num)
    

def helix_generate_track_from_param(param, theta=np.linspace(0, 2*np.pi, 360)):
    x0, y0, r0, m1, m2, m0 = param
    theta0 = np.arctan2(y0, x0)
    xx = -r0*np.cos(theta)
    yy = r0*np.sin(theta)
    x = xx*np.cos(theta0) - yy*np.sin(theta0)+x0
    y = xx*np.sin(theta0) + yy*np.cos(theta0)+y0
    
    r = np.sqrt(x**2 + y**2)
    z = m2*r**2 + m1*r + m0
    
    xyz = np.column_stack([x,y,z])
    return xyz
        
# calculate r/2r0
def calculate_r0_r_ratio(xyz, r0):
    xyz = xyz[np.argsort(np.fabs(xyz[:,2]))]
    x = xyz[:,0]
    y = xyz[:,1]
    r = np.sqrt(x**2 + y**2)
    ratio = 0.5*r/r0
    print('r:' + str(r))
    return ratio

# the angle we use for helix unrolling in dbscan
def helix_unroll_angle(xyz, ratio):
    xyz = xyz[np.argsort(np.fabs(xyz[:,2]))]
    x = xyz[:,0]
    y = xyz[:,1]
    angle = np.arctan2(y,x) - np.arccos(ratio)
    return angle
    
def study_all_helix_param():
       ## load an event ---
    event_id = '1003'
    data_dir  = '../../../input/train_100_events'
    csv_dir = '../../../input/'
    
    particles = pd.read_csv(os.path.join(data_dir, 'event00000%s-particles.csv'%event_id))
    hits      = pd.read_csv(os.path.join(data_dir, 'event00000%s-hits.csv' %event_id))
    truth     = pd.read_csv(os.path.join(data_dir, 'event00000%s-truth.csv'%event_id))
    
    truth = truth.merge(hits,       on=['hit_id'],      how='left')
    truth = truth.merge(particles,  on=['particle_id'], how='left')
    
    #--------------------------------------------------------
    df = truth.copy()
    p = df[['particle_id']].values.astype(np.int64)
    particle_ids = np.unique(p)
    r0_ratio_list = []    
    particle_ids = particle_ids[particle_ids!=0]
    for index, id in enumerate(particle_ids):
        t = df.loc[(df.particle_id==id)].as_matrix(columns=['x','y','z'])
        t = t[np.argsort(np.fabs(t[:,2]))]
        x0, y0, r0, m1, m2, m0  = helix_estimate_param_from_track(t)
        ratio = calculate_r0_r_ratio(t, r0)
        angle = helix_unroll_angle(t, ratio)
        r0_ratio_list.extend(ratio)
        print('r/2r0:' + str(ratio))
        
    
def study_helix_param():
    # Change this from 0 - 83xx :number of particles
    track_id = 1
     ## load an event ---
    event_id = '1003'

    data_dir  = '../../../input/train_100_events'
    csv_dir = '../../../input/'
    
    particles = pd.read_csv(os.path.join(data_dir, 'event00000%s-particles.csv'%event_id))
    hits      = pd.read_csv(os.path.join(data_dir, 'event00000%s-hits.csv' %event_id))
    truth     = pd.read_csv(os.path.join(data_dir, 'event00000%s-truth.csv'%event_id))
    
    truth = truth.merge(hits,       on=['hit_id'],      how='left')
    truth = truth.merge(particles,  on=['particle_id'], how='left')
    
    #--------------------------------------------------------
    df = truth.copy()
    p = df[['particle_id']].values.astype(np.int64)
    particle_ids = np.unique(p)
    particle_ids = particle_ids[particle_ids!=0]

    print('particle id:'+ str(particle_ids[track_id]))
    t = df.loc[(df.particle_id==particle_ids[track_id])].as_matrix(columns=['x','y','z'])
    t = t[np.argsort(np.fabs(t[:,2]))]
    param = helix_estimate_param_from_track(t)
    theta = helix_search_theta(t, param, S=1)
    x0, y0, r0, m1, m2, m0 = param
    
    ## Visualization only 
    helix = helix_generate_track_from_param(param, theta=theta)
    
    print(np.sqrt((t[:,0] - x0)**2 + (t[:,1] - y0)**2))
    
    
    fig1 = plt.figure(figsize=(8, 8))
    ax1 = fig1.add_subplot(111, projection='3d')

    ax1.plot(helix[:,0], helix[:,1], helix[:,2], '.-', color=[1,0,0], markersize=1)
    # ground truth
    ax1.plot(t[:,0], t[:,1], t[:,2], '.-', color=[0,1,0], markersize=8)
    
    
    ax1.set_title('xyz')
    ax1.set_xlabel('x (millimeters)')
    ax1.set_ylabel('y (millimeters)')
    ax1.set_zlabel('z (millimeters)')
    
    fig2 = plt.figure(figsize=(8, 8))
    ax2 = fig2.add_subplot(111)
    
    ax2.plot(helix[:,0], helix[:,1], '.-', color=[1,0,0], markersize=1)
    # ground truth
    ax2.plot(t[:,0], t[:,1], '.-', color=[0,1,0], markersize=8)
    
    ax2.set_title('XY projection')
    ax2.set_xlabel('x (millimeters)')
    ax2.set_ylabel('y (millimeters)')
    

if __name__ == '__main__':
    
    study_all_helix_param()


r:[ 31.93529352  71.84462371 116.60463047 259.81961184 390.67827821
 393.2940835  394.3463922  483.43435314 484.48910543 578.37196955
 691.19401468 817.64058923 954.8319061  953.78035222]
r/2r0:[0.00032261 0.00072576 0.00117792 0.00262466 0.00394658 0.00397301
 0.00398364 0.00488359 0.00489425 0.00584264 0.00698235 0.0082597
 0.00964559 0.00963496]
r:[  31.49445465   33.27312388  116.61050388   71.16033011   73.37991703
  172.60606348  260.23020568  359.53486951  503.37323667  659.36536364
  823.57127978 1023.82508823]
r/2r0:[0.00906236 0.00957416 0.03355404 0.020476   0.02111467 0.04966645
 0.07487982 0.10345419 0.1448429  0.18972878 0.23697813 0.29460007]
r:[ 32.45630357  71.26427526 116.66136058 171.35478222 262.29312211
 356.59830714 495.42366277 604.93485706 869.82571236 870.18850132]
r/2r0:[0.00261734 0.0057469  0.00940781 0.0138184  0.02115186 0.02875682
 0.03995199 0.0487832  0.07014454 0.0701738 ]
r:[ 32.06074252  33.47095191  73.16198381 115.55546281 117.66577697
 258.6838358

/home/nicole/anaconda3/envs/trackml/lib/python3.6/site-packages/ipykernel_launcher.py:77: RuntimeWarning: divide by zero encountered in true_divide
/home/nicole/anaconda3/envs/trackml/lib/python3.6/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in arccos


r:[124.66258722 259.06331626 360.14240218 499.6154234 ]
r/2r0:[0.20945468 0.43527113 0.60510146 0.83944023]
r:[173.04366479 116.17389065  72.49891941]
r/2r0:[0.13581977 0.09118341 0.05690348]
r:[71.23540878]
r/2r0:[inf]
r:[ 659.88722979  501.72246329  496.10807785  820.07198214  358.22500144
  262.4864712  1024.17075015 1018.88324259 1021.7123544 ]
r/2r0:[0.45584876 0.34658886 0.34271045 0.56650406 0.2474611  0.18132512
 0.70749507 0.70384247 0.70579682]
r:[ 820.20773885 1018.10514809]
r/2r0:[2.27764882 2.8271935 ]
r:[ 31.46087288  33.55297909  71.81737075 116.94935327 136.58096635
 160.06652472 291.20644372 348.60738928 414.26783693 487.85404691
 558.38528468 558.77092997]
r/2r0:[0.02378368 0.02536526 0.05429223 0.08841094 0.10325198 0.12100651
 0.22014518 0.26353894 0.31317668 0.36880612 0.42212606 0.4224176 ]
r:[32.22339943 73.00074473]
r/2r0:[0.78965574 1.7889316 ]
r:[73.00288132]
r/2r0:[inf]
r:[54.21641211 54.61758112 63.23674655 74.12812315]
r/2r0:[0.21597063 0.21756868 0.2519030

r:[ 31.46462159  72.36826473 118.3194452  138.3055302  162.27599726
 242.00659749 297.45729052 298.75249077 358.60137243 430.01483835
 510.67100523 590.79530256]
r/2r0:[0.00932195 0.02144037 0.03505421 0.04097544 0.04807711 0.0716987
 0.08812694 0.08851067 0.10624195 0.12739944 0.15129524 0.17503347]
r:[ 31.61338519  32.7783231   72.65287262 122.22546893 142.69009264
 167.24711622 249.26273393 305.54115352]
r/2r0:[0.00045059 0.0004672  0.00103554 0.00174211 0.0020338  0.00238382
 0.00355281 0.00435496]
r:[ 31.52413939  72.53732225 116.3100264  171.07049446 171.56733625
 263.33208897 361.41801088 498.90488363 505.05968782 659.51369307
 922.00236261]
r/2r0:[0.00080769 0.0018585  0.00298002 0.00438305 0.00439578 0.00674691
 0.00926001 0.0127826  0.01294029 0.01689761 0.02362291]
r:[ 31.63952127  73.22865753 115.76653077 170.79239194 172.64172602
 257.27765793 359.72946821 499.60189247 642.40386453 787.37850333
 952.41581651]
r/2r0:[0.0035227  0.00815317 0.01288927 0.01901577 0.01922167 0.

r:[  31.62402474   32.75365426   72.05243845  116.22645506  171.24326778
  256.51325012  356.05063282  362.42206697  499.09225133  660.01053474
  818.20200476 1018.72670644]
r/2r0:[0.01996907 0.02068238 0.0454977  0.07339149 0.108132   0.16197595
 0.22482909 0.22885235 0.31515309 0.41676535 0.51665576 0.6432776 ]
r:[  33.14089973   72.71443755  116.41476556  171.39904043  173.35420172
  256.81302036  356.31800936  362.66134871  497.29919496  663.09574481
  821.90360452 1017.28418389]
r/2r0:[0.00575504 0.01262714 0.02021586 0.02976409 0.03010361 0.04459655
 0.06187597 0.06297751 0.08635788 0.11514908 0.14272667 0.17665524]
r:[ 31.56414054  64.38207012  64.8644362   75.56156319  88.47452199
 118.44271778 118.8676663  140.4157638  161.75120295 273.13642643
 317.53693024]
r/2r0:[0.04586746 0.09355687 0.09425782 0.10980237 0.12856685 0.17211516
 0.17273268 0.20404532 0.23504893 0.39690849 0.46142913]
r:[ 32.46674953  71.43130855 116.93651694 136.62201155 160.11126477
 292.20086461 349.99258

...

r:[  31.96959302   72.34303335  115.40052369  117.26184908  170.87417439
  172.79663974  259.86465312  363.17745364  498.87841877  662.43778353
  817.17468808 1019.82267828]
r/2r0:[0.00392536 0.00888258 0.01416936 0.0143979  0.02098065 0.02121669
 0.03190727 0.04459245 0.06125438 0.08133688 0.10033612 0.12521809]
r:[  31.44338883   33.808557     71.26472092  115.76460043  172.12552773
  260.9442029   361.28305874  503.26953924  659.07395264  662.11081505
  817.75769277 1016.37893664]
r/2r0:[0.00424206 0.00456115 0.0096144  0.01561792 0.02322164 0.03520426
 0.04874108 0.06789662 0.08891636 0.08932607 0.11032455 0.13712075]
r:[  31.62944901   32.74425382   72.60376839  115.21772524  171.65554251
  256.45148644  262.84048803  357.96462507  502.35888824  658.58223037
  664.27564841  817.14154308 1020.46225518]
r/2r0:[0.02566025 0.02656466 0.05890177 0.0934735  0.1392602  0.2080532
 0.21323645 0.29040848 0.40755224 0.53429265 0.53891159 0.66292818
 0.8278776 ]
r:[ 47.09979443  47.42043694  

r:[ 31.47340264  33.39121366  72.36895778 116.42886191 171.5647106
 173.53037937 260.34985809 357.27735123]
r/2r0:[0.04420388 0.04689741 0.10164102 0.16352243 0.24095983 0.24372057
 0.36565711 0.50179019]
r:[ 31.44743566  33.70420172  71.65144706  73.84265505 116.38455304
 135.25589555 136.04119297 158.82086871 159.61119816 293.41710305
 353.43125155 423.95573058 505.97526209 587.10436285]
r/2r0:[0.01349221 0.01446045 0.03074133 0.03168145 0.04993362 0.05803018
 0.05836711 0.0681405  0.06847958 0.12588766 0.15163613 0.18189394
 0.2170836  0.25189123]
r:[ 31.83862228  71.29526397 126.96218262 148.4678671  174.2953571
 258.32130071 259.17716949 317.44129284 381.05785597 455.00081741
 535.70146006 614.71227907]
r/2r0:[0.01745132 0.03907821 0.06959025 0.0813779  0.09553441 0.14159054
 0.14205965 0.17399527 0.20886464 0.24939411 0.29362758 0.33693482]
r:[ 31.66248181  67.99789064  79.47851541  93.21674557 109.15447535
 125.06595902 147.6909895  170.32664148 243.18931066 286.26654976
 328.21

r:[1022.70936731  821.07896513]
r/2r0:[5.02485209 4.03418653]
r:[ 31.76930155  71.69853751 116.76233025 148.16719069 172.73322906
 261.91335644 299.43009335 367.83815131 442.80992766 526.1678836
 619.00419193]
r/2r0:[0.01283513 0.02896697 0.04717322 0.05986111 0.06978605 0.10581576
 0.12097292 0.1486105  0.17889989 0.21257739 0.25008424]
r:[ 91.6142728   92.22337105 106.90382678 107.5125093  125.19667684
 125.80120633 147.06692801 168.33143415 274.2280256  328.39118917
 388.77415248 450.20840385]
r/2r0:[0.05746495 0.057847   0.0670553  0.0674371  0.07852947 0.07890866
 0.09224756 0.10558569 0.17200921 0.20598299 0.24385813 0.28239269]
r:[ 58.38468919  58.82442241  68.12223399  79.77633493  93.45054564
 107.10852109 107.50630201 126.61626529 127.0056471  146.08141043
 146.46875247]
r/2r0:[0.59807542 0.60257992 0.69782394 0.81720509 0.95727964 1.09718789
 1.10126264 1.29701943 1.30100814 1.49641459 1.50038241]
r:[304.80029639]
r/2r0:[inf]
r:[ 32.96274843  71.73366827 121.9471584  122.756

r:[ 31.67578309  32.61963228  86.80557561  87.45263041 101.08120697
 118.25550842 118.82242072 138.19707935 158.11716699 257.40203415
 306.60882442 363.07101029 420.69115922]
r/2r0:[0.002591   0.0026682  0.00710048 0.00715341 0.00826819 0.00967301
 0.00971938 0.01130418 0.01293359 0.02105485 0.02507984 0.02969831
 0.03441149]
r:[ 32.5419539   72.09345683 116.40692306 171.09687096 256.16252829
 259.10160138 264.81362416 360.49290077 537.68647772 648.18971588
 766.07648609 890.40014071 892.4705195 ]
r/2r0:[0.02074568 0.04595999 0.07421007 0.10907522 0.16330506 0.16517873
 0.16882018 0.22981626 0.34277817 0.4132246  0.48837809 0.56763512
 0.568955  ]
r:[ 32.10806244  71.19031428 116.56913182 172.11905563 261.38616484
 358.632067   500.10477867 662.03693722 932.55123927]
r/2r0:[0.03368434 0.07468524 0.12229183 0.18056885 0.27421833 0.3762383
 0.52465629 0.69453814 0.97833273]
r:[  31.65783395   32.67170122   72.02479796  115.00753231  117.00855333
  172.35890236  257.7856662   356.15776407

r:[32.34361403]
r/2r0:[inf]
r:[256.55913931 262.54486256 360.69578995 498.49618237 503.21314677
 656.72557673 658.47456498 766.17800172]
r/2r0:[0.36762063 0.37619751 0.51683684 0.71428943 0.72104832 0.94101451
 0.94352062 1.09784763]
r:[500.58533156 660.27655147 664.93629785 361.00989095 262.5720953
 256.41312622 172.23132774 115.32581734  71.76496866  32.34414268]
r/2r0:[0.73141284 0.9647401  0.97154853 0.52747704 0.38364808 0.37464912
 0.25164981 0.1685043  0.10485689 0.04725852]
r:[ 32.34194494  71.08932733  72.96767514 115.45789654 117.45322985
 172.78551443 256.39717887 262.36289483 359.33574524 498.85326831
 501.41854068 657.41837102 765.6813402 ]
r/2r0:[0.05104299 0.11219524 0.1151597  0.182219   0.18536809 0.27269511
 0.40465346 0.41406873 0.56711409 0.78730469 0.79135327 1.03755673
 1.20842048]
r:[ 32.20220042  71.97282834 116.98844116 171.46879105 173.45151487
 256.8441562  358.72011042 364.58940359 535.44342059 638.61440402
 765.60414564 901.40022122 901.54164421]
r/2r0:[0.0